Imports

In [1]:
import pandas as pd
import numpy as np
import data_loader as m
import thermal_compensation as TC

User Inputs

In [2]:
machine = 'SwissDECO'

trainingNumber = 4

trainingDate = '2023.05.02'

errorName = 'X1-2'

bad_machineTemperatures_sensors = ['S1 - Casting','NC','Main Gang 3','Turret','S11 - Casting','Spindle Area','Main Gang 2','Turret - Back 2','S41 - Top', 'S41 - Rear 1', 'S41 - Rear 2', 'S41 - Bottom 1', 'S41 - Bottom 2', 'S41 - Radial Tool', 'S41 - T411', 'S41 - T414','S41 - T65', 'S41 - T68','Room Temperature', 'Cooling', 'S1 - Bottom']

uncorrelated_motorsTemperatures_sensors = ['S1', 'S11', 'S21', 'S4', 'S41']

ballScrewsNames = ['X1-BS','X2-BS','Z2-BS','Y4-BS','Z4-BS']

train_start = 0
train_end   = 399
pred_start  = train_end+1

e_threshold   = 10  # User requirements for accuracy
std_threshold = 25  # User tolerance for uncertainty
N_skip        = 2   # Improvement in efficiency by lowering number of measurements


Load Data

In [ ]:
errors = m.Measurement('errors',trainingNumber,trainingDate,machine)
errors.load_measurement(errorName,zero_normalize=True)
global_time = errors.get_global_time()

ballScrews = m.Measurement('errors',trainingNumber,trainingDate,machine)
ballScrews.load_measurement(ballScrewsNames,zero_normalize=True)

machineTemperatures = m.Measurement('machineTemperatures',trainingNumber,trainingDate,machine)
machineTemperatures.load_measurement(global_time=global_time,remove_sensors=bad_machineTemperatures_sensors,zero_normalize=True)

motorsTemperatures =  m.Measurement('motorsTemperatures',trainingNumber,trainingDate,machine)
motorsTemperatures.load_measurement(global_time=global_time,zero_normalize=True)


Smooth

In [4]:
errors.smooth()
machineTemperatures.smooth()
motorsTemperatures.smooth()

Create datasets

In [5]:
df_X = pd.concat((machineTemperatures.df,motorsTemperatures.df),axis=1)
df_y = errors.df.to_frame()*1000

Thermal Compensation

In [6]:
compensation = TC.Thermal_Compensation()
compensation.set_train_start(train_start)
compensation.set_train_end(train_end)
compensation.set_e_threshold(e_threshold)
compensation.set_std_threshold(std_threshold)
compensation.set_N_skip(N_skip)
compensation.append_data(df_X,df_y)

Train model

In [ ]:
compensation.train_model()

Predict

In [8]:
compensation.predict(recalibrations=True)

Plot result

In [ ]:
compensation.plot_result()